## *De novo* assembly 
This piece of code deals with the generation of contigs from the quality trimmed reads from the Illumina sequencer. It is a memory intensive process and will take quite some time, even with a powerful computing cluster at your fingertips. For assembly, two pieces of software are used:

* MegaHIT: https://github.com/voutcn/megahit
* QUAST: http://quast.sourceforge.net/

MegaHIT has quite a few options on how to perform the assembly, so it is recommended to familiarize yourself with their options and the way the software works.

#### MegaHIT
For MegaHIT, nothing beside the quality-trimmed reads from the quality control is required. Do realise that assembly is fairly sensitive to adapters still being present, so if this step is not yet performed, then this is a must before continuing. If MegaHIT exits without providing you an exit code, try re-starting the run with the second bit of code.

In [ ]:
#set parameters
SAMPLENAME=samplename
OUTDIR="$SAMPLENAME"_assembly
READSPATH=/path/to/reads/

[ -d "$READSPATH" ] || { echo 'Invalid path to reads, exiting.' && exit; }

#assign all reads from a single genome to a file
MCAV=$(cat "$OUTPUT_WORKING"/"$SAMPLENAME"_names.txt)
#this concatenates your reads into a single file
cat "$READSPATH"/*_R1_001_val*.fq.gz > "$READSPATH"/"$SAMPLENAME"_reads_R1_ALL.fastq
cat "$READSPATH"/*_R2_001_val*.fq.gz > "$READSPATH"/"$SAMPLENAME"_reads_R2_ALL.fastq

#run megahit
megahit --presets meta-large \
-1 "$READSPATH"/"$SAMPLENAME"_reads_R1_ALL.fastq \
-2 "$READSPATH"/"$SAMPLENAME"_reads_R2_ALL.fastq \
-o ../data/results/$OUTDIR --out-prefix $SAMPLENAME \
-t $NSLOTS \
-m 0.9 || { echo 'Error code 1: MegaHIT did not run, exiting.' && exit; }


Sometimes, MegaHIT will create 'bloated' assemblies from the metadata. This is often the case for complex microbiomes where some important metabolic partners are sequenced at low depth (MegaHIT needs about 9.2x coverage to accurately re-assemble the parts). There is no hard solution for this, but something that might work is normalizing the reads with BBNorm:

In [ ]:
#make sure Java version 7 or higher is active! Otherwise this will not work!
bbnorm.sh in=../../01_quality/data/results/ALL_reads_R1.fq out=../../01_quality/data/results/reads_norm/ALL_normalized_R1.fq target=70 min=5
bbnorm.sh in=../../01_quality/data/results/ALL_reads_R2.fq out=../../01_quality/data/results/reads_norm/ALL_normalized_R2.fq target=70 min=5

In [ ]:
#alternatively, this is also an option:
bbcms.sh mincount=2 highcountfraction=0.6

#### QUAST
QUAST executes a quality control for your contigs. This is also a good indicator whether a set of contigs is going to be of use to you. If there are very few large contigs (>1000), binning may not work very well. 

In [1]:
quast ../data/working/co_assembly_1/co-assembly_species.contigs.fa -o ../data/results/quast_output_co-assembly

SyntaxError: invalid syntax (<ipython-input-1-b9de1fa3c92f>, line 1)

### Mapping
Mapping is actually its own operation, regardless of what you do above. It is also, too little to warrant its own notebook. Mapping means re-aligning your reads back to your contigs using an aligner like Bowtie2, BWA, or something with the same funcitonalities. In this, it is important to always map the reads you used to the contigs that you generated (and later filtered). 

For binning, only contigs over 1000 basepairs can be considered. This is due to the fact that most binning softwares rely on the use of k-mers, and smaller sizes don't give an accurate k-mer signature for k-step =4. So, the following piece of code uses scripts from Anvi'O to do two things: fix the deflines of your contigs, so Anvi'O can use them (and makes them nice and uniform), and *remove* all contigs that are under 1000 bp. It is strongly recommended to save your contigs, so you can be sure to go back to them if you decide to go another direction with this. 

In [ ]:
#set parameters
SAMPLENAME=samplename
READSPATH=/path/to/reads/
CONTIGPATH=/path/to/contigs/
CONTIGFILE=contigfile.fa

[ -d $CONTIGPATH ] || { echo 'Invalid path to contigs, exiting.' && exit; }
[ -d $READSPATH ] || { echo 'Invalid path to contigs, exiting.' && exit; }

anvi-script-reformat-fasta $CONTIGPATH/$CONTIGFILE -o $CONTIGPATH/$SAMPLENAME.contigs-fixed.fa -l 1000 --simplify-names || { echo 'Exit code 1: unable to fix deflines, exiting.' && exit; }
#fixes deflines for later and filters on size (set to 1000 bp)

FIXEDCON="$SAMPLENAME".contigs-fixed.fa

bowtie2-build $CONTIGPATH/$FIXEDCON ../data/working/"$SAMPLENAME"_contigs || { echo 'Exit code 2: Unable to build Bowtie2-index, exiting.' && exit; }
#this builds an index of your contigs, which only needs to happen once

for f in <sample1> <sample2> <sample4> <sample5>
do
bowtie2 --threads $NSLOTS -x ../data/working/"$SAMPLENAME"_contigs -1 $READSPATH/"$f"_R1.fastq -2 $READSPATH/"$f"_R2.fastq -S ../data/working/"$f".sam
#this creates an alignment of your reads to your contigs and collects that in a .sam file

samtools view -F 4 -bS ../data/working/"$f".sam > ..data/working/"$f"-RAW.bam
#this converts your sam file to a bam file, but its neither sorted nor indexed, so we use an Anvi'O script to do so:

anvi-init-bam ../data/working/"$f"_-RAW.bam -o ../data/results/"$f".bam
#as said, this is how you index and sort your bam file

rm ../data/working/"$f"-RAW.bam
#to prevent confusion, remove the RAW file

done